In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("../lite_data/extracted_urls.csv", header=0)
data.head()

,Unnamed: 0,author,domain,count
0,0,NaN,aajtak.in,1
1,1,NaN,amarujala.com,1
2,2,NaN,aninews.in,1
3,3,NaN,apnampnews.com,1
4,4,NaN,bbc.co.uk,1


In [3]:
print(len(data))
print(len(data.dropna(subset=['author'])))

5940
5886


In [4]:
data = data.dropna(subset=['author'])

In [5]:
data.head()

,Unnamed: 0,author,domain,count
54,54,1234sangeeta1,abplive.com,1
55,55,1234sangeeta1,bbc.com,1
56,56,1234sangeeta1,hindustantimes.com,1
57,57,1234sangeeta1,indiatimes.com,1
58,58,1234sangeeta1,maharashtratimes.com,1


In [7]:
data.sample(5)

,Unnamed: 0,author,domain,count
5358,5358,rajeshchudasma,amritmahotsav.nic.in,3
663,663,BajpaiUsha,vikatan.com,1
1991,1991,ManishAnandINC,news18.com,1
1311,1311,HoneyJadejaBJP,firstpost.com,2
5194,5194,parmindercwc,pscp.tv,14


In [8]:
#map author and party handle
import glob
from collections import Counter, defaultdict

datafiles = glob.glob("/home/phadke/ONR/ONR/big_data/Twitter/*.csv")
author2party = defaultdict()

for d in datafiles:
    splitfilename = d.replace(".csv", "").split("_")
    party = splitfilename[len(splitfilename)-1]
    author = splitfilename[len(splitfilename)-2]
    author2party[author] = party
    

In [9]:
author2party

defaultdict(None,
            {'sanjaypaswanbjp': 'BJP',
             'manthanp96': 'BJP',
             'AhrariShahlaINC': 'INC',
             'satyaranjanbora': 'BJP',
             'idhawalbjp': 'BJP',
             'AgniChirag': 'BJP',
             'mohsinrazaul2': 'INC',
             'arunpat08567280': 'BJP',
             'AdvHrishikeshD': 'INC',
             'sevadalBAK': 'INC',
             'netikchoudhary': 'INC',
             'upadhya': 'BJP',
             'ShuklaPiyush001': 'INC',
             'jairohanionline': 'BJP',
             'GParadum': 'INC',
             'niravamin': 'BJP',
             'Ajaypra90327030': 'BJP',
             'BiswajitINC': 'INC',
             'jothims': 'INC',
             'ahir': 'BJP',
             'KoshtaTeekaram': 'INC',
             'amitpradhaan': 'BJP',
             'JaveedA51959947': 'INC',
             'Bjp4Chalisgaon': 'BJP',
             'bjpdoctorcellns': 'BJP',
             'atulguptabjp': 'BJP',
             'Rajendra7gurjar': 'INC',
     

In [10]:
data['party'] = data['author'].apply(lambda x: author2party[x])

In [11]:
data.sample(5)

,Unnamed: 0,author,domain,count,party
4757,4757,jothims,news7tamil.live,5,INC
3671,3671,ahir,angdaan.org,2,BJP
1321,1321,HoneyJadejaBJP,moneycontrol.com,4,BJP
205,205,AapkaBannaGupta,jharkhandnewsroom.com,1,INC
5199,5199,parmindercwc,theweek.in,1,INC


In [12]:
print(Counter(data[['author','party']].drop_duplicates()['party'].tolist()))

Counter({'BJP': 239, 'INC': 213})


### create co-sharing network

In [14]:
link_list = data.groupby("domain")['author'].apply(list).reset_index()
link_list.head()

,domain,author
0,1.book,"[BjpSanjayGupta, UmakantaSharma3]"
1,1.how,[Bharatgohil1174]
2,1.is,[AipcKochi]
3,1.ug,[abvphp]
4,11thadmission.org.in,[AdvHrishikeshD]


In [15]:
link_list['authorsize'] = link_list['author'].apply(lambda x: len(set(x)))
print(len(link_list))
print(len(link_list.loc[link_list['authorsize']>1]))

1440
426


In [37]:
multiple_shared_domains = link_list.loc[link_list['authorsize']>3]
multiple_shared_domains.head()

,domain,author,authorsize
16,aajtak.in,"[AjendraAjay, AkhileshLY, BBK, BJYM, Bilaliycc...",80
19,abplive.com,"[1234sangeeta1, AdvHrishikeshD, AjendraAjay, A...",76
21,abvp.org,"[ABVPMumbai, Ajaypra90327030, EshaanSG, RamNiw...",10
50,altnews.in,"[Banker, Bilaliyccg, Charanssapra, DrAmolDeshm...",13
52,amarujala.com,"[AlirazabilalB, BJPVirendrasinh, Bjp, Chetank7...",49


In [38]:
nodes = defaultdict()
edges = defaultdict()

for idx, row in multiple_shared_domains.iterrows():
    authorlist = list(set(row['author']))
    for a in authorlist:
        if a not in nodes:
            nodes[a] = 1
        else:
            nodes[a]+=1
            
    for i in range(len(authorlist)-1):
        for j in range(i+1, len(authorlist)):
            edgetuple = sorted([authorlist[i], authorlist[j]])
            if (edgetuple[0], edgetuple[1]) not in edges:
                edges[(edgetuple[0], edgetuple[1])] = 1
            else:
                edges[(edgetuple[0], edgetuple[1])] +=1
                
                
            

In [39]:
sorted(['imnitinmudgal','YogeshAttray'])

['YogeshAttray', 'imnitinmudgal']

In [40]:
edges

defaultdict(None,
            {('IYCWestBengal', 'imnitinmudgal'): 2,
             ('Bilaliyccg', 'imnitinmudgal'): 2,
             ('MYadavOfficial', 'imnitinmudgal'): 5,
             ('VidyaNandMishr8', 'imnitinmudgal'): 3,
             ('INCBhopal', 'imnitinmudgal'): 1,
             ('Rajendra7gurjar', 'imnitinmudgal'): 2,
             ('imnitinmudgal', 'jaldipchaudhar9'): 1,
             ('SachinPilot', 'imnitinmudgal'): 4,
             ('Sumankaushik27', 'imnitinmudgal'): 3,
             ('PunitTiwariINC', 'imnitinmudgal'): 3,
             ('imnitinmudgal', 'subratborah'): 1,
             ('chandanjnu', 'imnitinmudgal'): 4,
             ('harshyadavdeori', 'imnitinmudgal'): 2,
             ('BJYM', 'imnitinmudgal'): 3,
             ('imnitinmudgal', 'sureshdoshi42'): 1,
             ('imnitinmudgal', 'khare'): 2,
             ('Gulammustafa024', 'imnitinmudgal'): 2,
             ('Karunes60974736', 'imnitinmudgal'): 3,
             ('Inc', 'imnitinmudgal'): 3,
             ('DrVim

In [43]:
import networkx as nx

G = nx.Graph()

for n in nodes.keys():
    G.add_node(n, size=nodes[n], party=author2party[n])
    
for e in edges:
    if edges[e] > 2:
        G.add_edge(e[0], e[1], weight=edges[e])

In [44]:
nx.write_gexf(G, "./domainshares.gexf", encoding='utf-8')

### build common domain network

In [45]:
domlabel = data[['author','domain','party']].drop_duplicates()
domlabel.head()

,author,domain,party
54,1234sangeeta1,abplive.com,INC
55,1234sangeeta1,bbc.com,INC
56,1234sangeeta1,hindustantimes.com,INC
57,1234sangeeta1,indiatimes.com,INC
58,1234sangeeta1,maharashtratimes.com,INC


In [46]:
domain_by_party = domlabel.groupby(['domain','party']).size().reset_index()
domain_by_party.head()

,domain,party,0
0,1.book,BJP,2
1,1.how,INC,1
2,1.is,INC,1
3,1.ug,BJP,1
4,11thadmission.org.in,INC,1


In [47]:
domain_formated = pd.pivot_table(domain_by_party, values=0, index='domain', columns='party').reset_index()
domain_formated.head()

party,domain,BJP,INC
0,1.book,2.0,NaN
1,1.how,NaN,1.0
2,1.is,NaN,1.0
3,1.ug,1.0,NaN
4,11thadmission.org.in,NaN,1.0


In [48]:
domain_formated = domain_formated.fillna(0)

In [50]:
domain_formated.sample(5)

party,domain,BJP,INC
1277,theweek.in,4.0,9.0
1126,shrts.in,1.0,15.0
372,ex.pm,0.0,1.0
988,polytricks.in,0.0,1.0
1150,spr.ly,1.0,0.0


In [52]:
def prop_inclination(row):
    return float(row['BJP'])/float(row['BJP']+row['INC'])

domain_formated['domain_value'] = domain_formated.apply(lambda x: prop_inclination(x), axis=1)
domain_formated.head()

party,domain,BJP,INC,domain_value
0,1.book,2.0,0.0,1.0
1,1.how,0.0,1.0,0.0
2,1.is,0.0,1.0,0.0
3,1.ug,1.0,0.0,1.0
4,11thadmission.org.in,0.0,1.0,0.0


In [53]:
domain_formated.sample(5)

party,domain,BJP,INC,domain_value
658,kickex.com,0.0,1.0,0.000000
1049,rd.com,1.0,0.0,1.000000
79,asianetnews.com,10.0,5.0,0.666667
1040,rajnitinews.com,0.0,1.0,0.000000
623,justice.gov,0.0,1.0,0.000000


In [54]:
domainvalues = domain_formated.set_index("domain").to_dict()['domain_value']
domainvalues

{'1.book': 1.0,
 '1.how': 0.0,
 '1.is': 0.0,
 '1.ug': 1.0,
 '11thadmission.org.in': 0.0,
 '1drv.ms': 0.0,
 '1stbharatnews.com': 0.0,
 '2.help': 1.0,
 '2024.you': 0.0,
 '22scope.com': 0.0,
 '247wallst.com': 1.0,
 '30.am': 0.0,
 '4lakhdenahoga.com': 0.0,
 '4tvstream.in': 0.0,
 '99acres.com': 0.0,
 'aadarshhimachal.com': 0.0,
 'aajtak.in': 0.3125,
 'aankhondekhilive.in': 0.0,
 'abc.net.au': 1.0,
 'abplive.com': 0.4605263157894737,
 'absoluteindiahindi.com': 0.0,
 'abvp.org': 1.0,
 'acc.to': 1.0,
 'acntimes.com': 1.0,
 'adesiflava.com': 0.0,
 'adhikaraamcha.com': 1.0,
 'admnewsindia.com': 0.0,
 'adobe.com': 1.0,
 'advrishispeaks.blogspot.com': 1.0,
 'afr.com': 1.0,
 'afternoon.now': 1.0,
 'agniban.com': 0.0,
 'agniveer.com': 1.0,
 'aha.video': 0.0,
 'ahmedabadcity.gov.in': 1.0,
 'ahna.org.in': 1.0,
 'ai.gov.in': 1.0,
 'aiadmk.com': 0.0,
 'aiadmkachievements.com': 0.0,
 'aimamedia.org': 1.0,
 'airlive.net': 1.0,
 'aitatennis.com': 1.0,
 'aiwa4u.com': 0.0,
 'ajaishukla.blogspot.com': 0.0,
 '

In [55]:
domain_list = data.groupby("author")['domain'].apply(list).reset_index()
domain_list.head()

,author,domain
0,1234sangeeta1,"[abplive.com, bbc.com, hindustantimes.com, ind..."
1,13,"[cowin.gov.in, organiser.org, pib.gov.in]"
2,17,[infeed.in]
3,22,"[asianetnews.com, business-standard.com, busin..."
4,578deepesh,[pscp.tv]


In [60]:
domain_list['domainsize'] = domain_list['domain'].apply(lambda x: len(set(x)))
print(len(domain_list))
print(len(domain_list.loc[domain_list['domainsize']>1]))

452
394


In [65]:
multiple_shared_authors = domain_list.loc[domain_list['domainsize']>5]
multiple_shared_authors.head()

,author,domain,domainsize
0,1234sangeeta1,"[abplive.com, bbc.com, hindustantimes.com, ind...",8
3,22,"[asianetnews.com, business-standard.com, busin...",25
5,9,"[aje.io, amritmahotsav.nic.in, barandbench.com...",62
6,95,"[advrishispeaks.blogspot.com, bhaskar.com, ipl...",6
7,AAFAQUEGOHNA1,"[absoluteindiahindi.com, bbc.in, fortune.com, ...",11


In [66]:
nodes = defaultdict()
edges = defaultdict()

for idx, row in multiple_shared_authors.iterrows():
    authorlist = list(set(row['domain']))
    for a in authorlist:
        if a not in nodes:
            nodes[a] = 1
        else:
            nodes[a]+=1
            
    for i in range(len(authorlist)-1):
        for j in range(i+1, len(authorlist)):
            edgetuple = sorted([authorlist[i], authorlist[j]])
            if (edgetuple[0], edgetuple[1]) not in edges:
                edges[(edgetuple[0], edgetuple[1])] = 1
            else:
                edges[(edgetuple[0], edgetuple[1])] +=1


In [67]:
import networkx as nx

G = nx.Graph()

for n in nodes.keys():
    G.add_node(n, size=nodes[n], value=domainvalues[n])
    
for e in edges:
    if edges[e] > 5:
        G.add_edge(e[0], e[1], weight=edges[e])

In [68]:
nx.write_gexf(G, "./domainsims.gexf", encoding='utf-8')